# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="assets/LC_Memory_before.png" style="width:800px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****iIkA
LANGSMITH_API_KEY=****e049
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [9]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_mwD1TY8YuhiasuAx2Urc6TYL)
 Call ID: call_mwD1TY8YuhiasuAx2Urc6TYL
  Args:
    query: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_ReMFNfe1CEdXsvUsf74A1CZv)
 Call ID: call_ReMFNfe1CEdXsvUsf74A1CZv
  Args:
    query: SELECT * FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('table', 'Album', 'Album', 2, 'CREATE TABLE [Album]\n(\n    [AlbumId] INTEGER  NOT NULL,\n    [Title] NVARCHAR(160)  NOT NULL,\n    [ArtistId] INTEGER  NOT NULL,\n    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),\n    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)'), ('table', 'Artist', 'Artist', 3, 'CREATE TABLE [Artist]\n(\n    [ArtistId] INTEGER  NOT NULL,\n    [Name] NVARCHAR(120),\n    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])\n)'), ('table', 'Customer', 'Customer', 4, 'CREATE TABLE [Customer]\n(\n    [CustomerId] INTEGER  NOT NULL,\n    [FirstName] NVARCHAR(40)  NOT NULL

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_vVJfX5G7acPDLy7Ds3WcTTuV)
 Call ID: call_vVJfX5G7acPDLy7Ds3WcTTuV
  Args:
    query: SELECT i.Total FROM Invoice i JOIN Customer c ON i.CustomerId = c.CustomerId WHERE c.FirstName = 'Frank' AND c.LastName = 'Harris' ORDER BY i.InvoiceDate DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(5.94,)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

The total on your last invoice, Frank Harris, was $5.94.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [10]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_XXq6lVKyQiH8iZ7s8frybEgB)
 Call ID: call_XXq6lVKyQiH8iZ7s8frybEgB
  Args:
    query: SELECT title FROM your_table_name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: your_table_name
[SQL: SELECT title FROM your_table_name LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_ZwHPDpnqF89Y2uZiUygbnesH)
 Call ID: call_ZwHPDpnqF89Y2uZiUygbnesH
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_fhFhwvZYICC188uvMbJaWgmS)
 Call ID: call_fhFhwvZYICC188uvMbJaWgmS
  Args:
    query: SELECT name FROM Track LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('For Those About To Rock (We Salute You)',), ('Balls to the Wall',), ('Fast As a Shark',), ('Restless and Wild',), ('Princess of the Dawn',)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

The titles of the tracks are:

1. For Those About To Rock (We Salute You)
2. Balls to the Wall
3. Fast As a Shark
4. Restless and Wild
5. Princess of the Dawn


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## Add memory

In [11]:
from langgraph.checkpoint.memory import InMemorySaver

In [12]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [13]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_U9TezAYKVMKw9eotRJUnAztN)
 Call ID: call_U9TezAYKVMKw9eotRJUnAztN
  Args:
    query: SELECT name, sql FROM sqlite_master WHERE type='table' AND (
  lower(name) LIKE '%invoice%' OR lower(name) LIKE '%customer%' OR lower(name) LIKE '%client%' OR lower(name) LIKE '%user%' OR lower(name) LIKE '%person%' OR lower(name) LIKE '%contact%'
) ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Customer', 'CREATE TABLE [Customer]\n(\n    [CustomerId] INTEGER  NOT NULL,\n    [FirstName] NVARCHAR(40)  NOT NULL,\n    [LastName] NVARCHAR(20)  NOT NULL,\n    [Company] NVARCHAR(80),\n    [Address] NVARCHAR(70),\n    [City] NVARCHAR(40),\n    [State] NVARCHAR(40),\n    [Country] NVARCHAR(40),\n    [PostalCode]...'), ('Invoice', 'CREATE TABLE [Invoice]\n(\n    [InvoiceId] INTEGER  NOT NULL,\n    [CustomerId] INTEGER  NOT

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_xoHQyhB9fGpRcQjyE4Af8VQi)
 Call ID: call_xoHQyhB9fGpRcQjyE4Af8VQi
  Args:
    query: SELECT CustomerId, FirstName, LastName, Company, City, Country FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris' LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[(16, 'Frank', 'Harris', 'Google Inc.', 'Mountain View', 'USA')]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_pdJZOyko90ohxeEtiE97KNAK)
 Call ID: call_pdJZOyko90ohxeEtiE97KNAK
  Args:
    query: SELECT InvoiceId, InvoiceDate, Total FROM Invoice WHERE CustomerId = 16 ORDER BY InvoiceDate DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

Your most recent invoice (ID 374, dated 2013-07-04) totaled $5.94.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [14]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_3O1zOhNcAQJ2MikHxknvWg8N)
 Call ID: call_3O1zOhNcAQJ2MikHxknvWg8N
  Args:
    query: SELECT GROUP_CONCAT(Name, ' || ') AS Titles
FROM (
  SELECT t.Name AS Name
  FROM InvoiceLine il
  JOIN Track t ON t.TrackId = il.TrackId
  WHERE il.InvoiceId = 374
  ORDER BY il.InvoiceLineId
) AS x;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou || Through The Never || My Friend Of Misery || The Wait || Blitzkrieg || So What',)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

The tracks on that invoice were:
- Holier Than Thou
- Through The Never
- My Friend Of Misery
- The Wait
- Blitzkrieg
- So What


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


## Try your own queries
Now that there is memory, check the agents recall!

In [15]:
question = "what are the top 5 sold records from the db"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

what are the top 5 sold records from the db


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_RDy01nQqxzpiW0Ev1ykWoJal)
 Call ID: call_RDy01nQqxzpiW0Ev1ykWoJal
  Args:
    query: SELECT a.Title AS AlbumTitle,
       ar.Name AS ArtistName,
       SUM(il.Quantity) AS UnitsSold,
       ROUND(SUM(il.Quantity * il.UnitPrice), 2) AS GrossRevenue
FROM InvoiceLine il
JOIN Track t ON t.TrackId = il.TrackId
JOIN Album a ON a.AlbumId = t.AlbumId
JOIN Artist ar ON ar.ArtistId = a.ArtistId
GROUP BY a.AlbumId, a.Title, ar.Name
ORDER BY UnitsSold DESC, GrossRevenue DESC, a.Title
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Minha Historia', 'Chico Buarque', 27, 26.73), ('Greatest Hits', 'Lenny Kravitz', 26, 25.74), ('Unplugged', 'Eric Clapton', 25, 24.75), ('Acústico', 'Titãs', 22, 21.78), ('Greatest Kiss', 'Kiss', 20, 19.8)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

Top 5 sold albums:
- Minha Historia — Chico Buarque: 27 units, $26.73
- Greatest Hits — Lenny Kravitz: 26 units, $25.74
- Unplugged — Eric Clapton: 25 units, $24.75
- Acústico — Titãs: 22 units, $21.78
- Greatest Kiss — Kiss: 20 units, $19.80


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [17]:
question = "which one among them is the most expensive"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

which one among them is the most expensive


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_cI4kgyLeYZBKdegIVRFsHOLt)
 Call ID: call_cI4kgyLeYZBKdegIVRFsHOLt
  Args:
    query: WITH top5 AS (
  SELECT a.AlbumId,
         a.Title AS AlbumTitle,
         ar.Name AS ArtistName,
         SUM(il.Quantity) AS UnitsSold,
         SUM(il.Quantity * il.UnitPrice) AS GrossRevenue
  FROM InvoiceLine il
  JOIN Track t ON t.TrackId = il.TrackId
  JOIN Album a ON a.AlbumId = t.AlbumId
  JOIN Artist ar ON ar.ArtistId = a.ArtistId
  GROUP BY a.AlbumId, a.Title, ar.Name
  ORDER BY UnitsSold DESC, GrossRevenue DESC, a.Title
  LIMIT 5
), price AS (
  SELECT t5.AlbumId,
         t5.AlbumTitle,
         t5.ArtistName,
         ROUND(SUM(il.UnitPrice * il.Quantity) * 1.0 / SUM(il.Quantity), 2) AS AvgUnitPrice,
         MAX(il.UnitPrice) AS MaxUnitPrice
  FROM top5 t5
  JOIN Track tr ON tr.AlbumId = t5.AlbumId
  JOIN InvoiceLine il ON il.TrackId = tr.TrackId
  GROUP BY t5.AlbumId, t5.Alb

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

None—among those top 5, all tracks sell for $0.99, so the albums are priced the same per track. If you meant by total revenue, Minha Historia earned the most ($26.73).


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://eu.api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
